In [1]:
import os


os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/mlops/.ivy2/cache
The jars for the packages stored in: /home/mlops/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c7c90892-e91c-440e-9ef7-89a87886a13e;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 208ms :: artifacts dl 11ms
	:: modules in use:
	io.delta#delta-core_2.12;1.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      de

In [2]:
#reading silver data
ticketmaster_augmented = spark.read.format('delta').load('/home/mlops/project/DeltaLake/silver_data/ticketmaster_augmented_table')
ticketmaster_augmented.to_pandas_on_spark().head()

22/02/02 19:20:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/02/02 19:20:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,spotify_external_urls,spotify_followers,spotify_genres,spotify_href,spotify_id,spotify_images,spotify_name,spotify_popularity,spotify_type,spotify_uri,name,id,locale,distance,units,dates,classifications,_links,price_min,price_max,_embedded
0,(https://open.spotify.com/artist/21aa4pj9BvbFB...,"(254135,)","[german pop, pop]",https://api.spotify.com/v1/artists/21aa4pj9Bvb...,21aa4pj9BvbFB2iT8kRpnq,"[(640, https://i.scdn.co/image/ab6761610000e5e...",Michael Schulte,68.0,artist,spotify:artist:21aa4pj9BvbFB2iT8kRpnq,Michael Schulte,Z698xZC2Z17uFO-,en-us,81.96,MILES,"{'start': {'dateTime': '2022-09-27T18:00:00Z',...","[{'subGenre': None, 'genre': None, 'family': F...",{'self': {'href': '/discovery/v2/events/Z698xZ...,29.0,29.0,"{'venues': [{'country': '{name=Germany, countr..."
1,(https://open.spotify.com/artist/2l1xTBTIBaAiJ...,"(26241,)",[german indie folk],https://api.spotify.com/v1/artists/2l1xTBTIBaA...,2l1xTBTIBaAiJhekoLM8mk,"[(640, https://i.scdn.co/image/ab6761610000e5e...",Jules Ahoi,49.0,artist,spotify:artist:2l1xTBTIBaAiJhekoLM8mk,Jules Ahoi,Z698xZC2Z178kCa,en-us,81.96,MILES,"{'start': {'dateTime': '2022-10-14T18:00:00Z',...","[{'subGenre': None, 'genre': None, 'family': F...",{'self': {'href': '/discovery/v2/events/Z698xZ...,18.0,18.0,"{'venues': [{'country': '{name=Germany, countr..."
2,(https://open.spotify.com/artist/2tppd6KkhK4UL...,"(95017,)",[irish rock],https://api.spotify.com/v1/artists/2tppd6KkhK4...,2tppd6KkhK4ULAd217Ecq1,"[(640, https://i.scdn.co/image/ab6761610000e5e...",The Coronas,48.0,artist,spotify:artist:2tppd6KkhK4ULAd217Ecq1,The Coronas,Z698xZC2Z17uzjy,de-de,81.96,MILES,"{'start': {'dateTime': '2022-10-29T18:00:00Z',...","[{'subGenre': None, 'genre': None, 'family': F...",{'self': {'href': '/discovery/v2/events/Z698xZ...,22.0,22.0,"{'venues': [{'country': '{name=Germany, countr..."
3,(https://open.spotify.com/artist/0VUeSHZjbneQ8...,"(82494,)","[german pop, german singer-songwriter]",https://api.spotify.com/v1/artists/0VUeSHZjbne...,0VUeSHZjbneQ8Ti7whkL9A,"[(640, https://i.scdn.co/image/ab6761610000e5e...",Joel Brandenstein,57.0,artist,spotify:artist:0VUeSHZjbneQ8Ti7whkL9A,Joel Brandenstein,Z698xZC2Z17uIa9,en-us,81.79,MILES,"{'start': {'dateTime': '2023-03-08T19:00:00Z',...","[{'subGenre': None, 'genre': None, 'family': F...",{'self': {'href': '/discovery/v2/events/Z698xZ...,36.5,45.5,"{'venues': [{'country': '{name=Germany, countr..."
4,(https://open.spotify.com/artist/7qXzy6c5RWT0X...,"(2454686,)",[german pop],https://api.spotify.com/v1/artists/7qXzy6c5RWT...,7qXzy6c5RWT0XlVQcOBIDG,"[(640, https://i.scdn.co/image/ab6761610000e5e...",Mark Forster,67.0,artist,spotify:artist:7qXzy6c5RWT0XlVQcOBIDG,Mark Forster,Z698xZC2Z17uPF9,de-de,82.34,MILES,"{'start': {'dateTime': '2022-09-10T17:00:00Z',...","[{'subGenre': None, 'genre': None, 'family': F...",{'self': {'href': '/discovery/v2/events/Z698xZ...,40.0,49.0,"{'venues': [{'country': '{name=Germany, countr..."


In [3]:
spotify_flattened = ticketmaster_augmented

spotify_flattened = spotify_flattened.withColumn("spotify_url", ticketmaster_augmented.spotify_external_urls.spotify)
spotify_flattened = spotify_flattened.withColumn("spotify_followers", ticketmaster_augmented.spotify_followers.total)
spotify_flattened = spotify_flattened.withColumn("spotify_image_url", ticketmaster_augmented.spotify_images[0].url)

spotify_flattened = spotify_flattened.drop('spotify_external_urls', 'spotify_images')
spotify_flattened.show()


+-----------------+--------------------+--------------------+--------------------+--------------------+------------------+------------+--------------------+--------------------+---------------+------+--------+-----+--------------------+--------------------+--------------------+---------+---------+--------------------+--------------------+--------------------+
|spotify_followers|      spotify_genres|        spotify_href|          spotify_id|        spotify_name|spotify_popularity|spotify_type|         spotify_uri|                name|             id|locale|distance|units|               dates|     classifications|              _links|price_min|price_max|           _embedded|         spotify_url|   spotify_image_url|
+-----------------+--------------------+--------------------+--------------------+--------------------+------------------+------------+--------------------+--------------------+---------------+------+--------+-----+--------------------+--------------------+-------------------

In [4]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import regexp_replace

ticketmaster_flattened = spotify_flattened

ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_date", ticketmaster_flattened.dates['start']['localDate'])
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_name", ticketmaster_flattened._embedded['venues'][0]['name'])
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_id", ticketmaster_flattened._embedded['venues'][0]['id'])
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_url", ticketmaster_flattened._embedded['venues'][0]['url'])
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_postal_code", ticketmaster_flattened._embedded['venues'][0]['postalCode'])
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_timezone", ticketmaster_flattened._embedded['venues'][0]['timezone'])
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_city", ticketmaster_flattened._embedded['venues'][0]['city'])
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_city", regexp_replace('ticketmaster_venue_city', '\{name=', ""))
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_city", regexp_replace('ticketmaster_venue_city', '}', ""))
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_country", ticketmaster_flattened._embedded['venues'][0]['country'])
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_country", regexp_replace('ticketmaster_venue_country', '\{name=', ""))
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_country", regexp_replace('ticketmaster_venue_country', ',.+', ""))
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_address", ticketmaster_flattened._embedded['venues'][0]['address'])
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_address", regexp_replace('ticketmaster_venue_address', '\{line1=', ""))
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_address", regexp_replace('ticketmaster_venue_address', '}', ""))
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_url", ticketmaster_flattened._embedded['attractions'][0].url)
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_artist_locale", ticketmaster_flattened._embedded['attractions'][0].locale)
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_latitude", ticketmaster_flattened._embedded['venues'][0]['location'])
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_longitude", regexp_replace('ticketmaster_venue_latitude', '.+longitude=', ""))
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_longitude", regexp_replace('ticketmaster_venue_longitude', '}', ""))
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_latitude", regexp_replace('ticketmaster_venue_latitude', '.+latitude=', ""))
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_latitude", regexp_replace('ticketmaster_venue_latitude', ', longitude=.+', ""))
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_latitude", ticketmaster_flattened['ticketmaster_venue_latitude'].cast(DoubleType()))
ticketmaster_flattened = ticketmaster_flattened.withColumn("ticketmaster_venue_longitude", ticketmaster_flattened['ticketmaster_venue_longitude'].cast(DoubleType()))


ticketmaster_flattened = ticketmaster_flattened.drop('dates', 'classifications', '_links', '_embedded')

ticketmaster_flattened.show()

+-----------------+--------------------+--------------------+--------------------+--------------------+------------------+------------+--------------------+--------------------+---------------+------+--------+-----+---------+---------+--------------------+--------------------+-----------------+-----------------------+---------------------+----------------------+------------------------------+---------------------------+-----------------------+--------------------------+--------------------------+--------------------+--------------------------+---------------------------+----------------------------+
|spotify_followers|      spotify_genres|        spotify_href|          spotify_id|        spotify_name|spotify_popularity|spotify_type|         spotify_uri|                name|             id|locale|distance|units|price_min|price_max|         spotify_url|   spotify_image_url|ticketmaster_date|ticketmaster_venue_name|ticketmaster_venue_id|ticketmaster_venue_url|ticketmaster_venue_postal_cod

In [5]:

ticketmaster_renamed = ticketmaster_flattened

ticketmaster_renamed = ticketmaster_renamed.withColumnRenamed('name', 'ticketmaster_name')
ticketmaster_renamed = ticketmaster_renamed.withColumnRenamed('id', 'ticketmaster_id')
ticketmaster_renamed = ticketmaster_renamed.withColumnRenamed('locale', 'ticketmaster_locale')
ticketmaster_renamed = ticketmaster_renamed.withColumnRenamed('distance', 'ticketmaster_distance')
ticketmaster_renamed = ticketmaster_renamed.withColumnRenamed('units', 'ticketmaster_units')
ticketmaster_renamed = ticketmaster_renamed.withColumnRenamed('price_min', 'ticketmaster_price_min')
ticketmaster_renamed = ticketmaster_renamed.withColumnRenamed('price_max', 'ticketmaster_price_max')


ticketmaster_renamed.show()

+-----------------+--------------------+--------------------+--------------------+--------------------+------------------+------------+--------------------+--------------------+---------------+-------------------+---------------------+------------------+----------------------+----------------------+--------------------+--------------------+-----------------+-----------------------+---------------------+----------------------+------------------------------+---------------------------+-----------------------+--------------------------+--------------------------+--------------------+--------------------------+---------------------------+----------------------------+
|spotify_followers|      spotify_genres|        spotify_href|          spotify_id|        spotify_name|spotify_popularity|spotify_type|         spotify_uri|   ticketmaster_name|ticketmaster_id|ticketmaster_locale|ticketmaster_distance|ticketmaster_units|ticketmaster_price_min|ticketmaster_price_max|         spotify_url|   spot

In [6]:

ticketmaster_renamed.write.format("delta").mode("overwrite").save("/home/mlops/project/DeltaLake/gold_data/ticketmaster_flattened_table")


In [3]:
ticketmaster = spark.read.format('delta').load('/home/mlops/project/DeltaLake/gold_data/ticketmaster_flattened_table')
ticketmaster.to_pandas_on_spark().head()



,spotify_followers,spotify_genres,spotify_href,spotify_id,spotify_name,spotify_popularity,spotify_type,spotify_uri,ticketmaster_name,ticketmaster_id,ticketmaster_locale,ticketmaster_distance,ticketmaster_units,ticketmaster_price_min,ticketmaster_price_max,spotify_url,spotify_image_url,ticketmaster_date,ticketmaster_venue_name,ticketmaster_venue_id,ticketmaster_venue_url,ticketmaster_venue_postal_code,ticketmaster_venue_timezone,ticketmaster_venue_city,ticketmaster_venue_country,ticketmaster_venue_address,ticketmaster_url,ticketmaster_artist_locale,ticketmaster_venue_latitude,ticketmaster_venue_longitude
0,NaN,None,None,None,None,NaN,None,None,SEB MELLIA,rZ7SnyZ1AduEvs,fr-fr,66.01,MILES,25.0,39.0,None,None,2023-02-04,PALAIS DES CONGRES-SALLE ERASME,rZFSnyZeek6,https://www.ticketmaster.fr/fr/salle/palais-de...,67000,Europe/Paris,Strasbourg,France,PLACE DE BORDEAUX - WACKEN,None,None,48.598322,7.760900
1,66848.0,"[album rock, mellow gold, new wave pop, philly...",https://api.spotify.com/v1/artists/7uhvDINTTiD...,7uhvDINTTiD0XBrP9fquN1,The Hooters,56.0,artist,spotify:artist:7uhvDINTTiD0XBrP9fquN1,The Hooters,Z698xZC2Z17uPao,en-us,67.49,MILES,35.0,35.0,https://open.spotify.com/artist/7uhvDINTTiD0XB...,https://i.scdn.co/image/ab6761610000e5eb15896b...,2022-06-28,Musiktheater REX,Z698xZC2Za7r9,https://www.ticketmaster.de/venue/musiktheater...,64625,Europe/Berlin,Bensheim,Germany,Fabrikstraße 10,http://www.ticketmaster.de/artist/918988,en-us,49.680740,8.619640
2,760031.0,"[francoton, pop urbaine]",https://api.spotify.com/v1/artists/7gU9VyFRN3J...,7gU9VyFRN3JWPJ5oHOil60,Tayc,76.0,artist,spotify:artist:7gU9VyFRN3JWPJ5oHOil60,TAYC,rZ7SnyZ1Ad700d,fr-fr,67.72,MILES,26.0,26.0,https://open.spotify.com/artist/7gU9VyFRN3JWPJ...,https://i.scdn.co/image/ab6761610000e5ebce9a6f...,2021-12-16,LA LAITERIE,rZFSnyZeeFa,https://www.ticketmaster.fr/fr/salle/la-laiter...,67000,Europe/Paris,Strasbourg,France,"13,rue du Hohwald",http://www.ticketmaster.fr/artist/1044372,en-us,48.576090,7.729777
3,876137.0,"[electro swing, nu jazz]",https://api.spotify.com/v1/artists/37J1PlAkhRK...,37J1PlAkhRK7yrZUtqaUpQ,Caravan Palace,66.0,artist,spotify:artist:37J1PlAkhRK7yrZUtqaUpQ,CARAVAN PALACE,rZ7SnyZ1Adk7qv,fr-fr,67.72,MILES,34.2,34.2,https://open.spotify.com/artist/37J1PlAkhRK7yr...,https://i.scdn.co/image/ab6761610000e5ebdedbe7...,2021-12-18,LA LAITERIE,rZFSnyZeeFa,https://www.ticketmaster.fr/fr/salle/la-laiter...,67000,Europe/Paris,Strasbourg,France,"13,rue du Hohwald",http://www.ticketmaster.fr/artist/141183,en-us,48.576090,7.729777
4,139871.0,[cali rap],https://api.spotify.com/v1/artists/1grN0519h2z...,1grN0519h2zYqpRtYbDZAl,Larry June,67.0,artist,spotify:artist:1grN0519h2zYqpRtYbDZAl,LARRY,rZ7SnyZ1AduOGE,fr-fr,67.72,MILES,26.0,26.0,https://open.spotify.com/artist/1grN0519h2zYqp...,https://i.scdn.co/image/ab6761610000e5eb1c0b98...,2022-01-15,LA LAITERIE,rZFSnyZeeFa,https://www.ticketmaster.fr/fr/salle/la-laiter...,67000,Europe/Paris,Strasbourg,France,"13,rue du Hohwald",None,None,48.576090,7.729777
